In [ ]:
from numpy.random import seed
seed(8)
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection 
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split as tts

%matplotlib inline

In [ ]:
iris_data = datasets.load_iris()

In [ ]:
def read_in_and_split_data(iris_data):
    X = iris_data['data']
    Y = iris_data['target']
    train_data, test_data, train_targets, test_targets = tts(X,Y, test_size = 0.1)
    return (train_data, test_data, train_targets, test_targets)   

In [ ]:
iris_data = datasets.load_iris()
train_data, test_data, train_targets, test_targets = read_in_and_split_data(iris_data)

In [ ]:
train_targets = tf.keras.utils.to_categorical(np.array(train_targets))
test_targets = tf.keras.utils.to_categorical(np.array(test_targets))

In [ ]:
train_data.shape

In [ ]:
def compile_model(model):
    optim = tf.keras.optimizers.Adam(learning_rate = 0.0001)
    model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics = ['accuracy'])  

In [ ]:
def train_model(model, train_data, train_targets, epochs):
    history = model.fit(train_data, train_targets,validation_split = 0.2, epochs = epochs)
    return history

In [ ]:
def get_model(input_shape, dropout_rate, weight_decay):

    model = Sequential()
    model.add(Dense(64, kernel_initializer= 'he_uniform', bias_initializer = 'ones',kernel_regularizer = tf.keras.regularizers.l2(weight_decay), input_shape = (4,)))
    model.add(Dense(128,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dense(128,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dense(128,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(64,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dense(64,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dense(64,kernel_regularizer = tf.keras.regularizers.l2(weight_decay), activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    return model

    

In [ ]:
model = get_model(train_data[0].shape, 0.3, 0.001)

In [ ]:
compile_model(model)

In [ ]:
history = train_model(model, train_data, train_targets, epochs=800)

In [ ]:
import pandas as pd
df1 = pd.DataFrame(history.history)
df1.head()

In [ ]:
df1.plot(y = ['accuracy', 'val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')

In [ ]:
df1.plot(y = ['loss', 'val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')

In [ ]:
def get_callbacks():
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30, mode = 'min')
    learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.2, patience = 20)
    return (early_stopping, learning_rate_reduction)
    

In [ ]:
call_model = get_model(train_data[0].shape, 0.3, 0.0001)
compile_model(call_model)
early_stopping, learning_rate_reduction = get_callbacks()
call_history = call_model.fit(train_data, train_targets, epochs=800, validation_split=0.15,
                         callbacks=[early_stopping, learning_rate_reduction], verbose=10)

In [ ]:
df = pd.DataFrame(call_history.history)
df.head()

In [ ]:
df.plot(y = ['accuracy','val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')

In [ ]:
df.plot(y = ['loss', 'val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Loss vs Epochs')

In [ ]:
test_loss, test_acc = call_model.evaluate(test_data, test_targets, verbose=0)
print("Test loss: {:.3f}\nTest accuracy: {:.2f}%".format(test_loss, 100 * test_acc))